In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [2]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201025


In [3]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [4]:
#ディレクトリ指定
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_スコアカード")

In [13]:
#input
input_file = "2020年8月度_承認口座数.xlsx"
input_book = pd.ExcelFile(data_dir/ f"{input_file}")

In [55]:
print(input_file)

2020年8月度_承認口座数.xlsx


In [35]:
#各シート名を取得
input_sheet_name = input_book.sheet_names
print(input_sheet_name)

['フォーマット', '3', '4', '5', '6', '7', '8', '2020上']


In [19]:
#シート数を確認
num_sheet = len(input_sheet_name)
print(num_sheet)

8


In [44]:
#最初の10行を確認
input_sheet_df = input_book.parse(input_sheet_name[1])
input_sheet_df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,2020年3月度,大丸松坂屋カード・大丸松坂屋お得意様ゴールドカード 承認口座数（本月）,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ＪＦＲカード㈱,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-07 00:00:00,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,大丸松坂屋百貨店,NaN,NaN,NaN,NaN,NaN,NaN,関係店,NaN,法人,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,心斎橋店,梅田店,京都店,神戸店,東京店,札幌店,名古屋店,上野店,静岡店,高槻店,豊田店,下関店,大松合計,高知大丸,下関大丸,定借,営業,オンライン,総合計
4,承認口座数,大丸松坂屋カード,一般VISA,NaN,49,114,193,221,200,127,109,119,39,14,10,1717,2912,0,NaN,5,66,283,3266
5,NaN,NaN,一般Master,NaN,7,18,34,66,68,23,21,14,2,0,5,115,373,0,NaN,1,18,82,474
6,NaN,NaN,ゴールドVISA,NaN,4,2,6,20,11,7,8,11,1,0,1,31,102,0,NaN,0,1,13,116
7,NaN,NaN,ゴールドMaster,NaN,1,1,2,7,2,2,3,1,0,0,0,3,22,0,NaN,0,4,10,36
8,NaN,NaN,お得意様ゴールドVISA,NaN,143,0,48,167,191,82,162,19,20,0,0,14,846,5,NaN,0,0,0,851
9,NaN,NaN,お得意様ゴールドMaster,NaN,30,0,16,48,92,44,55,8,8,0,0,2,303,0,NaN,0,0,0,303


In [32]:
#最後の5行確認
input_sheet_df.tail(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,2020年3月度,大丸松坂屋カード・大丸松坂屋お得意様ゴールドカード 承認口座数（本月）,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
56,NaN,NaN,NaN,NaN,心斎橋店,梅田店,京都店,神戸店,東京店,札幌店,名古屋店,上野店,静岡店,高槻店,豊田店,下関大丸,大松合計,高知大丸,下関大丸,法人･定借,法人･営業,ｵﾝﾗｲﾝ,総合計
57,NaN,NaN,ゴールドVM切替数 計,NaN,81,61,47,132,39,57,113,14,14,8,16,0,582,NaN,0,0,0,0,582
58,NaN,NaN,NaN,前 年,11,17,25,43,12,13,16,1,5,0,0,0,143,0,0,0,0,0,143
59,NaN,NaN,NaN,増 減 率,6.364,2.588,0.88,2.07,2.25,3.385,6.063,∞,1.8,--,--,--,3.07,--,--,--,--,--,3.07
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,旧銀座店の切替は上野店に含む,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
#余計な行と列を削除
input_sheet_df = input_book.parse(input_sheet_name[1],
                                  skiprows =4,
                                  skipfooter = 1,
                                 usecols = [2,22])

In [71]:
#余計な行を削除
input_sheet_df = input_sheet_df.iloc[0:6,:]

In [72]:
#列名を変更
input_sheet_df = input_sheet_df.rename(columns={"Unnamed: 2":"brand",
                                                "総合計":"no_of_approval"})

In [73]:
input_sheet_df

,brand,no_of_approval
0,一般VISA,3266
1,一般Master,474
2,ゴールドVISA,116
3,ゴールドMaster,36
4,お得意様ゴールドVISA,851
5,お得意様ゴールドMaster,303


In [74]:
#シート名からmonthの列を生成
input_sheet_df['month'] = input_book.sheet_names[1]

In [75]:
#ファイル名を取得
input_sheet_df["file_name"] = input_file

In [76]:
#ファイル名からyearの列を生成
input_sheet_df["year"] = input_sheet_df["file_name"].str[:4]

In [78]:
#一時的に生成した列を削除
input_sheet_df = input_sheet_df.drop(columns = {"file_name"})

In [82]:
input_sheet_df

,brand,no_of_approval,month,year
0,一般VISA,3266,3,2020
1,一般Master,474,3,2020
2,ゴールドVISA,116,3,2020
3,ゴールドMaster,36,3,2020
4,お得意様ゴールドVISA,851,3,2020
5,お得意様ゴールドMaster,303,3,2020


In [83]:
input_sheet_df = input_sheet_df.reindex(columns=["year","month","brand","no_of_approval"])

In [84]:
input_sheet_df

,year,month,brand,no_of_approval
0,2020,3,一般VISA,3266
1,2020,3,一般Master,474
2,2020,3,ゴールドVISA,116
3,2020,3,ゴールドMaster,36
4,2020,3,お得意様ゴールドVISA,851
5,2020,3,お得意様ゴールドMaster,303


In [ ]:
#他のシートもまとめて

In [115]:
output_df = pd.DataFrame(index=[], columns=[])
for i in range(1,7):
    #余計な行と列を削除
    input_sheet_df = input_book.parse(input_sheet_name[i],
                                  skiprows =4,
                                  skipfooter = 1,
                                 usecols = [2,22])
    #余計な行を削除
    input_sheet_df = input_sheet_df.iloc[0:10,:]
    input_sheet_df = input_sheet_df.drop(7)
    
    input_sheet_df = input_sheet_df.replace({'VISA　(グループ※1)': "一般VISA", 'Master(グループ※2)': '一般Master'})
    #列名を変更
    input_sheet_df = input_sheet_df.rename(columns={"Unnamed: 2":"brand",
                                                "総合計":"no_of_approval"})
    #シート名からmonthの列を生成
    input_sheet_df['month'] = input_book.sheet_names[i]
    #ファイル名を取得
    input_sheet_df["file_name"] = input_file
    #ファイル名からyearの列を生成
    input_sheet_df["year"] = input_sheet_df["file_name"].str[:4]
    #一時的に生成した列を削除
    input_sheet_df = input_sheet_df.drop(columns = {"file_name"})
    
    #以上シートごとに処理したデータフレームを一つのデータフレームに統合
    output_df = pd.concat([output_df,input_sheet_df])
    
    #列の並べ替え
    output_df = output_df.reindex(columns=["year","month","brand","no_of_approval"])    
    

In [116]:
output_df

,year,month,brand,no_of_approval
0,2020,3,一般VISA,3266
1,2020,3,一般Master,474
2,2020,3,ゴールドVISA,116
3,2020,3,ゴールドMaster,36
4,2020,3,お得意様ゴールドVISA,851
5,2020,3,お得意様ゴールドMaster,303
6,2020,3,臨時(即発),481
8,2020,3,一般VISA,1
9,2020,3,一般Master,3
0,2020,4,一般VISA,676


In [117]:
output_df.groupby(["year","month"])["no_of_approval"].sum()

year  month
2020  3        5531
      4        1405
      5        1456
      6        2967
      7        4193
      8        3839
Name: no_of_approval, dtype: int64

In [118]:
output_df.groupby(["year","month","brand"])["no_of_approval"].sum()

year  month  brand         
2020  3      お得意様ゴールドMaster     303
             お得意様ゴールドVISA       851
             ゴールドMaster          36
             ゴールドVISA           116
             一般Master           477
             一般VISA            3267
             臨時(即発)             481
      4      お得意様ゴールドMaster     137
             お得意様ゴールドVISA       350
             ゴールドMaster          15
             ゴールドVISA            35
             一般Master           103
             一般VISA             676
             臨時(即発)              89
      5      お得意様ゴールドMaster     123
             お得意様ゴールドVISA       352
             ゴールドMaster          10
             ゴールドVISA            24
             一般Master           202
             一般VISA             705
             臨時(即発)              40
      6      お得意様ゴールドMaster     235
             お得意様ゴールドVISA       775
             ゴールドMaster          20
             ゴールドVISA            83
             一般Master           320
             一般VISA            1196


In [6]:
print(yyyymmdd)

20200101


新規Vintage CUSTデータの読込＆集計

In [8]:
df = pd.read_excel(data_dir /f"{input_file}",encoding='shift-jis')

In [26]:
df = pd.read_excel(data_dir /f"{input_file}",
                  dtype={'カード加入日':int,
                        'カード提携会社1CD':str,
                        'カード提携会社2CD':str,
                        '切替CD':str,
                        '提携カード特殊エリア':str,
                        '受付部支店':str,
                        'リボ支払方法CD':str,
                        'エンボス内容':str,
                        'カード種類CD':str,
                        '入会申込番号':str,
                        '提携先顧客番号1':str},encoding='shift-jis')
df = df[df['カード加入日'] >= yyyymmdd]

In [27]:
df.head()

,カード加入日,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,再発行CD,旧カード種類CD,旧カードブランドCD,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,入会申込番号,提携先顧客番号1
0,20200301,0,D00,50,31,0,0,8,106,0,NaN,NaN,12,4000,20,06000079005,1,155193037597925,NaN
1,20200301,0,D00,50,31,0,0,8,103,0,NaN,NaN,12,3000,20,03000914305,1,155193037597644,NaN
2,20200301,0,D00,50,31,0,0,8,170,0,NaN,NaN,12,10000,20,10000872121,1,155193037597610,NaN
3,20200301,0,D00,50,31,0,0,8,102,0,NaN,NaN,12,2000,20,02000914214,1,155193037597081,NaN
4,20200301,0,D00,50,31,0,0,8,106,0,NaN,NaN,12,6000,20,06000401014,1,155193037597719,NaN


In [28]:
print(df.dtypes)

カード加入日          int32
続柄              int64
カード提携会社1CD     object
カード提携会社2CD     object
カード種類CD        object
カードブランドCD       int64
切替CD           object
勧誘CD            int64
受付部支店          object
再発行CD           int64
旧カード種類CD      float64
旧カードブランドCD    float64
リボ支払方法CD       object
勧誘団体CD          int64
提携カード特殊エリア     object
エンボス内容         object
カード年会費CD        int64
入会申込番号         object
提携先顧客番号1       object
dtype: object


In [29]:
df = df.rename(columns={'カード加入日':'kanyu_date',
                        '続柄':'zokugara',
                        'カード提携会社1CD':'teikei_1',
                        'カード提携会社2CD':'teikei_2',
                        '切替CD':'kirikae',
                        '提携カード特殊エリア':'tokushu',
                        '受付部支店':'uketuke',
                        'カード種類CD':'card',
                        '入会申込番号':'applino',
                        '提携先顧客番号1':'campaign_cd'})
df.head()

,kanyu_date,zokugara,teikei_1,teikei_2,card,カードブランドCD,kirikae,勧誘CD,uketuke,再発行CD,旧カード種類CD,旧カードブランドCD,リボ支払方法CD,勧誘団体CD,tokushu,エンボス内容,カード年会費CD,applino,campaign_cd
0,20200301,0,D00,50,31,0,0,8,106,0,NaN,NaN,12,4000,20,06000079005,1,155193037597925,NaN
1,20200301,0,D00,50,31,0,0,8,103,0,NaN,NaN,12,3000,20,03000914305,1,155193037597644,NaN
2,20200301,0,D00,50,31,0,0,8,170,0,NaN,NaN,12,10000,20,10000872121,1,155193037597610,NaN
3,20200301,0,D00,50,31,0,0,8,102,0,NaN,NaN,12,2000,20,02000914214,1,155193037597081,NaN
4,20200301,0,D00,50,31,0,0,8,106,0,NaN,NaN,12,6000,20,06000401014,1,155193037597719,NaN


In [30]:
print(df['campaign_cd'].unique())

[nan '4980132204845009' '6900109524082515' ... '4980132306299006'
 '4980682053187002' '5279253344018005']


In [32]:
q = """
        SELECT
        kanyu_date
        ,case 
            when substr(tokushu,1,2) = "20" then '店頭タブレット'
            when uketuke = "299" and substr(campaign_cd,1,2) = "10" then 'オンライン(百貨店送客)'
            when uketuke = "299" then 'オンライン'
            when uketuke in ("200","001") then '郵送'
            else '店頭紙' 
            end as uketsuke_channel
        ,case 
            when kirikae = "0" then 'N/A'
            when substr(tokushu,1,2) = "80" then 'DM'
            when substr(tokushu,1,2) = "81" then "コンタクト"
            when substr(tokushu,1,2) = "86" then "店頭"
            else "店頭"
            end as kirikae_channel
        ,case
            when teikei_2 =50 then '臨時'
            when teikei_1 ="D02" then '外商'
            when teikei_2 ="05" then 'ハウスゴールド'
            when card ="02"  then 'ゴールド'
            else '一般'
            end as card_shubetsu
        ,case 
            when zokugara = 0 then '本人'
            else '家族' 
            end as kazoku_shubetsu
        ,sum(case when kirikae = "0" then 1 else 0 end ) as no_of_new
        ,sum(case when kirikae = "1" then 1 else 0 end ) as no_of_kirikae
        FROM 
          df
        where
          teikei_2 not in ("95") 
        group by
        1,2,3,4,5
        """

In [33]:
df_sql = sqldf(q, locals())
df_sql['kanyu_date']=pd.to_datetime(df_sql['kanyu_date'], format='%Y%m%d')
df_sql['kanyu_year'] = df_sql['kanyu_date'].dt.year
df_sql['kanyu_month'] = df_sql['kanyu_date'].dt.month
df_sql['kanyu_day'] = df_sql['kanyu_date'].dt.day
df_sql['kanyu_date'] = df_sql['kanyu_date'].dt.strftime('%Y/%m/%d')
df_sql.head()

,kanyu_date,uketsuke_channel,kirikae_channel,card_shubetsu,kazoku_shubetsu,no_of_new,no_of_kirikae,kanyu_year,kanyu_month,kanyu_day
0,2020/03/01,店頭タブレット,N/A,臨時,本人,12,0,2020,3,1
1,2020/03/02,オンライン,N/A,ゴールド,本人,2,0,2020,3,2
2,2020/03/02,オンライン,N/A,一般,本人,25,0,2020,3,2
3,2020/03/02,店頭タブレット,N/A,ゴールド,家族,2,0,2020,3,2
4,2020/03/02,店頭タブレット,N/A,ゴールド,本人,3,0,2020,3,2


CSVアウトプット

In [34]:
df_sql.to_csv(data_dir /f"{output_file}.csv", index=None,header=True,encoding='utf-8-sig')